In [1]:
%cd ..

/workspaces/Code


In [2]:
%matplotlib inline

from __future__ import annotations
from dataclasses import dataclass  

import numpy as np
import tensorflow as tf
import os
from IPython.display import display, HTML
import math
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO
import base64
import statistics
from datetime import datetime
from image_provider import ImageProvider
from map_provider import MapProvider, ImageProjection
from vector import Vector2D
from WeightCalculators import AbsModelBasedWeightCalculator, ModelBasedWeightCalculator
from ModelBuilders import MobileNetBuilder
from tensorflow.keras import mixed_precision
import json
from WeightCalculators.Transformers import BaseTransformer, StretchedTanTransformer
from color_generator import ColorGenerator
from util import hex_color_dump

In [3]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)


def enable_gpu_memory_growth():
    """
    Enables memory growth mode for GPUs.
    """
    gpus = tf.config.experimental.list_physical_devices('GPU')
    assert len(gpus) > 0, "No GPUs detected!"
            
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

enable_gpu_memory_growth()

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 2080, compute capability 7.5


2022-05-07 18:20:24.226452: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-07 18:20:24.229724: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-07 18:20:24.229871: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-07 18:20:24.230239: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [4]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [5]:
current_date = datetime.now().strftime("%Y-%m-%d.%H-%M-%S")
run_folder = f"Data/Offsets/{current_date}"

## Run Config Data

In [6]:
base_size = 224
crop_scale = 3
drone_crop_delta = 0

crop_size = math.floor(base_size * crop_scale)
print(f"Drone crop size: {crop_size + drone_crop_delta}")
print(f"City crop size: {crop_size}")

graph_size = 1500
target_distances = [1, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 75, 100, 150, 250, 350, 500, 700, 1000]
images_threshold = 0.66

# model_builder = EfficientNetB2Builder(second_dense=8, weight_file="enB2-e8-s59-b1-rloss.h5")
# model_builder = ResNetBuilder(second_dense=8, weight_file="resnet50-e1-s80-b1-rloss.h5")
model_builder = MobileNetBuilder(second_dense=8, weight_file="trainall-d64-e4-s35-b1-rloss.h5")

Drone crop size: 672
City crop size: 672


In [7]:
city_image = ImageProvider("City/NewCut/City_2017.jpg")
drone_image = ImageProvider("City/NewCut/City_2016.jpg")

projection = None # ImageProjection(Vector2D(8000, 8000), Vector2D(6117, 5281))

In [8]:
def generate_offset_points(offset: int):
    vector_offsets = [
        Vector2D(offset, 0),
        Vector2D(0, offset),
        Vector2D(-offset, 0),
        Vector2D(0, -offset)
    ]

    def round_up(number):
        floored = math.floor(number)
        return floored if number - floored < 0.5 else floored + 1

    offset = round_up(math.sqrt((offset ** 2) / 2))
    side_offsets = [
        Vector2D(offset, offset),
        Vector2D(offset, -offset),
        Vector2D(-offset, offset),
        Vector2D(-offset, -offset)
    ]

    vector_offsets.extend(side_offsets)

    return vector_offsets

In [9]:
class ReferencePoint:
    def __init__(self, point: Vector2D, label: str = ""):
        self.point = point
        self.label = label

    def __str__(self):
        """Human-readable string representation of the vector."""

        return f'{self.label} - ({self.point.x}, {self.point.y})'

    def __repr__(self):
        """Unambiguous string representation of the vector."""

        return self.__str__()

def generate_reference_points():
    reference_points = [
        ReferencePoint(Vector2D(5881, 1656), 'forest'),
        ReferencePoint(Vector2D(15650, 24030), 'forest'),
        ReferencePoint(Vector2D(8650, 2000), 'forest'),
        ReferencePoint(Vector2D(5846, 4752), 'apartments'),
        ReferencePoint(Vector2D(8046, 19828), 'apartments'),
        ReferencePoint(Vector2D(3330, 9590), 'apartments'),
        ReferencePoint(Vector2D(5310, 7960), 'apartments'),
        ReferencePoint(Vector2D(2501, 3402), 'apartments'),
        ReferencePoint(Vector2D(1973, 22699), 'field'),
        ReferencePoint(Vector2D(4780, 4160), 'road'),
        ReferencePoint(Vector2D(14130, 1500), 'road'),
        ReferencePoint(Vector2D(24230, 10280), 'river'),
        ReferencePoint(Vector2D(9960, 16340), 'small buildings'),
        ReferencePoint(Vector2D(7781, 5645), 'forest & road'),
        ReferencePoint(Vector2D(25000, 24570), 'forest & road'),
        ReferencePoint(Vector2D(9250, 20580), 'apartments & roads'),
        ReferencePoint(Vector2D(25678, 13083), 'apartments & small buildings'),
        ReferencePoint(Vector2D(18543, 9141), 'apartments & forest'),
        ReferencePoint(Vector2D(3298, 25260), 'road & field'),
        ReferencePoint(Vector2D(2013, 3788), 'road & small buildings'),
    ]

    return reference_points

### Generate random locations

In [10]:
def generate_random_points():
    for x, y in city_map.generate_random_locations(10):
        x += 1000 if x < 1500 else 0
        x -= 1000 if x > 25000 else 0

        y += 1000 if y < 1500 else 0
        y -= 1000 if y > 25000 else 0

        print(f"ReferencePoint(Vector2D({x}, {y}), ''),")

# generate_random_points()

### Review Anchor Data

In [11]:
def show_reference_points():
    for reference_point in generate_reference_points():
        display(f"Showcase for point: {reference_point}")
        anchor =  drone_map.get_cropped_image(reference_point.point.x, reference_point.point.y)

        display(Image.fromarray(anchor))

# show_reference_points()

## Setup Objects

In [12]:
city_map = MapProvider(
    image_provider=city_image,
    crop_size=crop_size,
    projection=projection)

drone_map = MapProvider(
    image_provider=drone_image,
    crop_size=crop_size + drone_crop_delta,
    projection=projection)

In [13]:
model_calculator = AbsModelBasedWeightCalculator(
    model_builder,
    batch_size=64, 
    transformer=StretchedTanTransformer(images_threshold))

2022-05-07 18:20:59.231064: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-07 18:20:59.234686: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-07 18:20:59.234886: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-07 18:20:59.235016: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

## Run Data Dump/Save

In [14]:
def asBase64(image_np):
    image = Image.fromarray(image_np)
    
    buffered = BytesIO()
    image.save(buffered, format="JPEG")
    img_str = base64.b64encode(buffered.getvalue()).decode('ascii')

    return img_str

In [15]:
def dump_results(anchor, offset_data: OffsetData):
    display("----------------------------------------------")

    display(f"Offset: {offset_data.vector_offset}.")
    display(f"Distance: {offset_data.real_distance}")

    display(f"Weight: {offset_data.weight}.")
    display(f"Delta: {offset_data.delta:.4f}")


    images = HTML(f"""
        <div class="row">
            <img src=data:image/jpeg;base64,{asBase64(anchor)} style="height:300px"/>
            <img src=data:image/jpeg;base64,{asBase64(offset_data.offset_image)} style="height:300px"/>
        </div>
        """)

    display(images)

In [16]:
@dataclass
class OffsetData:
    reference_point: ReferencePoint
    vector_offset: Vector2D
    original_weight: float
    weight: float
    offset_image: np.ndarray

    def __post_init__(self):
        self.delta = self.weight - self.original_weight
        self.real_distance =  self.reference_point.point.distance_to(self.reference_point.point + self.vector_offset)

    def dump(self):
        return {
            "offset": f"{self.vector_offset}",
            "real_distance": self.real_distance,
            "weight": self.weight,
            "delta": self.delta,
        }

@dataclass
class DistanceData:
    target_distance: int
    offsets: list[OffsetData]

    def __post_init__(self):
        self.avg_delta = statistics.mean([offset.delta for offset in self.offsets])
        self.avg_weight = statistics.mean([offset.weight for offset in self.offsets])
        self.offsets_dump = [offset.dump() for offset in self.offsets]

    def dump(self):
        return {
            "target_distance": f"{self.target_distance}",
            "avg_weight": self.avg_weight,
            "avg_delta": self.avg_delta,
            "offsets": self.offsets_dump
        }

@dataclass
class PointData:
    reference_point: ReferencePoint
    original_weight: float
    anchor_image: np.ndarray
    distances: list[DistanceData]

    def __post_init__(self):
        self.avg_delta = statistics.mean([distance.avg_delta for distance in self.distances])
        self.distances_dump = [distance.dump() for distance in self.distances]

    def dump(self):
        return {
            "label": self.reference_point.label,
            "reference_point": f"{self.reference_point.point}",
            "original_weight": self.original_weight,
            "avg_delta": self.avg_delta,
            "distances": self.distances_dump
        }

# Experiment steps

In [17]:
def run():
    reference_points = generate_reference_points()

    point_dump = []
    for reference_point in reference_points:
        display(f"TEST for point: {reference_point}")

        anchor =  drone_map.get_cropped_image(reference_point.point.x, reference_point.point.y)
        positive = city_map.get_cropped_image(reference_point.point.x, reference_point.point.y)
        weights = model_calculator.create_normalized_weights_from_images(anchor, [positive])
        original_weight = weights[0]

        display(f"Initial positive weight: {original_weight}")

        distances_dump = []
        for distance in target_distances:
            vector_offsets = generate_offset_points(distance)

            offset_points = [reference_point.point + vector_offset for vector_offset in vector_offsets]
            offset_particles = [(offset_point.x, offset_point.y) for offset_point in offset_points]

            offset_images = city_map.create_images_from_particles_threaded(offset_particles, 2)
            weights = model_calculator.create_normalized_weights_from_images(anchor, offset_images)

            offsets_dump = []
            for (weight, vector_offset, offset_image) in zip(weights, vector_offsets, offset_images):
                offset_data = OffsetData(
                    reference_point = reference_point,
                    vector_offset = vector_offset,
                    original_weight = original_weight,
                    weight = weight,
                    offset_image = offset_image
                )
                
                offsets_dump.append(offset_data)
                #dump_results(anchor, offset_data)

            distance_data = DistanceData(target_distance=distance, offsets=offsets_dump)
            distances_dump.append(distance_data)

        point_data = PointData(
            reference_point=reference_point,
            original_weight=original_weight,
            anchor_image=anchor,
            distances=distances_dump)
        
        point_dump.append(point_data)

    return point_dump


In [18]:
point_dump = run()

'TEST for point: forest - (5881, 1656)'

2022-05-07 18:21:01.940966: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100


'Initial positive weight: 0.8146882738385881'

'TEST for point: forest - (15650, 24030)'

'Initial positive weight: 0.9023764084796516'

...

'Initial positive weight: 0.7539762574799207'

'TEST for point: road & field - (3298, 25260)'

'Initial positive weight: 0.9306931787607621'

'TEST for point: road & small buildings - (2013, 3788)'

'Initial positive weight: 0.8881467313182597'

## Dump Data

In [19]:
label_colors = {
    'apartments': '#8e44ad',
    'road': '#34495e',
    'small buildings': '#c0392b',
    'field': '#f1c40f',
    'forest': '#16a085',
    'river': '#3498db',
}

for label, color in label_colors.items():
    hex_color_dump(color, label)

color_generator = ColorGenerator("#2c3e50", label_colors)

<span style="font-family: monospace">apartments <span style="color: #8e44ad">████</span></span>

<span style="font-family: monospace">road <span style="color: #34495e">████</span></span>

<span style="font-family: monospace">small buildings <span style="color: #c0392b">████</span></span>

<span style="font-family: monospace">field <span style="color: #f1c40f">████</span></span>

<span style="font-family: monospace">forest <span style="color: #16a085">████</span></span>

<span style="font-family: monospace">river <span style="color: #3498db">████</span></span>

In [20]:
class AggregatedData:
    def __init__(self, target_distance: int, distances_dump: list[DistanceData]):
        self.target_distance = target_distance
        self.avg_weight = statistics.mean([distance_data.avg_weight for distance_data in distances_dump])
        self.avg_delta = statistics.mean([distance_data.avg_delta for distance_data in distances_dump])
        self.min_weight = min([distance_data.avg_weight for distance_data in distances_dump])

    def dump(self):
        return {
            "target_distance": self.target_distance,
            "avg_weight": self.avg_weight,
            "avg_delta": self.avg_delta,
            "min_weight": self.min_weight,
        }

In [21]:
class PointGraphGenerator:
    def __init__(self, label: str, transformer: BaseTransformer, y_ticks: list[float], separator_line: float, colored):
        self.label = label
        self.distances = [0] + target_distances
        self.target_max = max(target_distances)
        self.separator_line = separator_line
        self.y_ticks = y_ticks
        self.colored = colored

        self.transformer = transformer

    def _generate_graph_base(self):
        fig, ax = plt.subplots()
        fig.set_size_inches(16, 5)

        ax.set_xlabel("Distance (px)")
        ax.set_ylabel("Weights")

        ax.set_xlim([0, self.target_max + 1])
        ax.set_xticks(np.arange(0, self.target_max + 1, 25))

        if self.y_ticks is not None:
            ax.set_yticks(self.y_ticks)

        if self.separator_line is not None:
            ax.plot([0, self.target_max], [self.separator_line, self.separator_line], color='r', alpha=0.8)

        return fig, ax

    def generate_distance_graph(self, data: PointData):
        fig, ax = self._generate_graph_base()
        transformer = self.transformer

        weight_data = [transformer.transform(data.original_weight)] + [transformer.transform(distance.avg_weight) for distance in data.distances]
        ax.plot(self.distances, weight_data, marker="o")

        return fig, ax

    def generate_aggregated_graph(self, point_dump: list[PointData], aggregated_dump: list[AggregatedData]):
        fig, ax = self._generate_graph_base()
        transformer = self.transformer

        for point_data in point_dump:
            if self.colored:
                label = point_data.reference_point.label
                data_color = color_generator.label_to_color(point_data.reference_point.label)
                alpha = 0.40
            else:
                label = "runs"
                data_color = "k"
                alpha = 0.15

            weight_data = [transformer.transform(point_data.original_weight)] + [transformer.transform(distance.avg_weight) for distance in point_data.distances]
            ax.plot(self.distances, weight_data, color=data_color, alpha=alpha, label=label)

        average_reference_weight = statistics.mean([point_data.original_weight for point_data in point_dump])
        total_weight_data = [transformer.transform(average_reference_weight)] + [transformer.transform(aggregate.avg_weight) for aggregate in aggregated_dump]
        
        ax.plot(self.distances, total_weight_data, marker='o', label="avg")

        handles, labels = ax.get_legend_handles_labels()

        by_label = dict(zip(labels, handles))
        labels = [f"{label} x{labels.count(label)}" if label != "avg" else label for label in by_label.keys()]

        lgd = ax.legend(by_label.values(), labels, loc='center left', bbox_to_anchor=(1, 0.5))

        return fig, ax, lgd

In [22]:
class PointDataSaver:
    def __init__(self, colored_graphs=False):
        self.linear_graph_generator = PointGraphGenerator("linear", BaseTransformer(), np.arange(0, 1.01, 0.05), images_threshold, colored_graphs)
        self.log_graph_generator = PointGraphGenerator("log", StretchedTanTransformer(images_threshold), np.arange(0, 100, 5), 0, colored_graphs)

    def _build_point_folder(self, point: Vector2D):
        point_dir = f"{run_folder}/{point}"
        os.makedirs(point_dir, exist_ok=True)

        return point_dir

    def _save_stats(self, point_folder: str, data: PointData):
        point_file = f"{point_folder}/point-stats.json"

        with open(point_file, "w") as file:
            file.write(json.dumps(data.dump(), indent=4))

    def _save_images(self, point_folder: str, data: PointData):
        images_folder = f"{point_folder}/images"
        os.makedirs(images_folder, exist_ok=True)

        Image.fromarray(data.anchor_image).save(f"{images_folder}/anchor.jpg")

        for distance in data.distances:
            distance_folder = f"{images_folder}/{distance.target_distance}"
            os.makedirs(distance_folder, exist_ok=True)

            for offset in distance.offsets:
                file_name = f"{distance_folder}/{offset.vector_offset}.jpg"
                Image.fromarray(offset.offset_image).save(file_name)

    def _save_individual_graphs(self, graph_generator: PointGraphGenerator, point_folder: str, data: PointData):
        fig, ax = graph_generator.generate_distance_graph(data)
        fig.savefig(f"{point_folder}/{graph_generator.label}_regular_graph.png", bbox_inches='tight', facecolor='w', dpi=300)

        ax.set_xlim([0, 150])
        ax.set_xticks(np.arange(0, 151, 5))

        fig.savefig(f"{point_folder}/{graph_generator.label}_zoomed_graph.png", bbox_inches='tight', facecolor='w', dpi=300)

        plt.close(fig)

    def _save_normalized_graph(self, point_folder: str, data: PointData):
        normalized_fig, ax = self.linear_graph_generator.generate_distance_graph(data)

        ax.set_ylim([0, 1.01])
        ax.set_yticks(np.arange(0, 1.01, 0.05))

        normalized_fig.savefig(f"{point_folder}/linear_normalized_graph.png", bbox_inches='tight', facecolor='w', dpi=300)

        plt.close(normalized_fig)

    def _save_individual(self, data: PointData):
        point_folder = self._build_point_folder(data.reference_point)
        
        self._save_stats(point_folder, data)
        self._save_images(point_folder, data)
        
        self._save_individual_graphs(self.linear_graph_generator, point_folder, data)
        self._save_individual_graphs(self.log_graph_generator, point_folder, data)
        self._save_normalized_graph(point_folder, data)

    def _get_aggregated_dump(self, point_dump: list[PointData]) -> list[AggregatedData]:
        aggregated_distance_data = {}
        for target_distance in target_distances:
            aggregated_distance_data[target_distance] = []
            
        for point_data in point_dump:
            for distance_data in point_data.distances:
                aggregated_distance_data[distance_data.target_distance].append(distance_data)

        aggregated_dump =  [AggregatedData(target_distance, dumps) for target_distance, dumps in aggregated_distance_data.items()]

        return aggregated_dump

    def _save_aggregated(self, point_dump: list[PointData], aggregated_dump: list[AggregatedData]):
        all_run_data = {
            "threshold": images_threshold,
            "avg_reference_weight": statistics.mean([point_data.original_weight for point_data in point_dump]),
            "targets": [aggregate.dump() for aggregate in aggregated_dump],
        }

        aggregate_file = f"{run_folder}/run-stats.json"

        with open(aggregate_file, "w") as file:
            file.write(json.dumps(all_run_data, indent=4))

    def _save_aggregated_graphs(self, graph_generator: PointGraphGenerator, point_dump: PointData, aggregated_dump: list[AggregatedData]):
        fig, ax, lgd = graph_generator.generate_aggregated_graph(point_dump, aggregated_dump)
        fig.savefig(f"{run_folder}/{graph_generator.label}_regular_graph.png", bbox_extra_artists=(lgd,), bbox_inches='tight', facecolor='w', dpi=300)

        ax.set_xlim([0, 150])
        ax.set_xticks(np.arange(0, 151, 5))

        fig.savefig(f"{run_folder}/{graph_generator.label}_zoomed_graph.png", bbox_extra_artists=(lgd,), bbox_inches='tight', facecolor='w', dpi=300)

        plt.close(fig)

    def save_all_individual(self, point_dump: list[PointData]):
        for point_data in point_dump:
            self._save_individual(point_data)

    def save_all_aggregated(self, point_dump: list[PointData]):
        aggregated_dump =  self._get_aggregated_dump(point_dump)
        self._save_aggregated(point_dump, aggregated_dump)
        
        self._save_aggregated_graphs(self.linear_graph_generator, point_dump, aggregated_dump)
        self._save_aggregated_graphs(self.log_graph_generator, point_dump, aggregated_dump)

point_data_saver = PointDataSaver(colored_graphs=True)

In [23]:
point_data_saver.save_all_individual(point_dump)

In [24]:
point_data_saver.save_all_aggregated(point_dump)